# ACG Seminar Assignment -- Neural Volume Compression  

Student: Anže Kristan  
class: Advanced Computer Graphics  
semester: Spring 2023/24  
school: Faculty of Computer and Information Science, University of Ljubljana  

#### imports and setup:

In [21]:
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
import pandas as pd
import torch
import torch.nn as nn
from torchsummary import summary
import numpy as np
import seaborn as sns
import pandas as pd
from scipy.ndimage import gaussian_filter
from os import path as ospath

In [23]:
folder_path = None #todo_add_path
files = {
    "body": {
        "file_name": "body_512x512x226_1x1x1_uint8.raw",
        "shape": [512, 512, 226],
        "dtype": np.uint8
    }
}

# from https://www.geeksforgeeks.org/reading-binary-files-in-python/

# Open the file in binary mode
def readRawFile(file_path, datatype, shape, doReshape=True, divideBy=2**8, gaussianFilter=False, cutoffLow=(0,0), cutoffHigh=(255,255)):
  array = np.fromfile(file_path, dtype=datatype).astype(np.float32) # read from file based on datatype
  array[array<cutoffLow[0]] = cutoffLow[1] # set values below cutoff[0] to the value of cutoff[1]
  array[array>cutoffHigh[0]] = cutoffHigh[1]
  array = np.divide(array, divideBy) # divide input by specified (to get input into range 0-1)
  if (doReshape):
    array = np.reshape(array, list(reversed(shape))).transpose() # need to make sure reshape takes the same order as the file all x->all y->all z
    if (gaussianFilter):
      array = gaussian_filter(array, sigma=0.1, radius=3)
  return array

#### Dataset class:

Dataset class to help with machine learning; sample it like an array with an index (int) and returns a sample corresponding to that index

In [24]:
class Dataset:
  def __init__(self,samples,stride=[4,4,4],kernel_size=[8,8,8], returnSurroundingIndices=True):
    self.stride = stride # [x,y,z]
    self.kernel_size = kernel_size # single value treated as length of the side of a cube
    self.returnSurroundingIndices = returnSurroundingIndices
    self.orig_shape = samples.shape
    self.samples = self.pad_to_divisible_and_kernel(samples)

    self.unfold_dims = np.array([ # num samples in each dimension
        int(np.ceil((self.orig_shape[0] - self.kernel_size[0])/self.stride[0])+1),
        int(np.ceil((self.orig_shape[1] - self.kernel_size[1])/self.stride[1])+1),
        int(np.ceil((self.orig_shape[2] - self.kernel_size[2])/self.stride[2])+1)
    ])
    self.length =  np.prod(self.unfold_dims) # total num samples

  def pad_to_divisible_and_kernel(self, arr):
    # pad the array to be divisible by kernel size
    to_pad = np.remainder(arr.shape, self.kernel_size)
    to_pad = [self.kernel_size[x] - to_pad[x] if to_pad[x] > 0 else 0 for x in range(3)]
    arr_pd = np.pad(arr, ( (0, to_pad[0]), (0, to_pad[1]), (0, to_pad[2]) ), mode='constant', constant_values='0')

    if self.returnSurroundingIndices:
      # pad an extra kernel_size around array
      extra_pad = [[self.kernel_size[x],self.kernel_size[x]] for x in range(3)]
      arr_pd2 = torch.tensor(np.pad(arr_pd, extra_pad, mode='constant', constant_values=0))
    else:
      arr_pd2 = arr_pd
    return torch.Tensor(arr_pd2)


  def __getitem__(self,index):
    arr_coord = np.multiply(np.unravel_index(index, self.unfold_dims), self.stride)
    if self.returnSurroundingIndices:
      arr_coord = np.add(arr_coord, self.kernel_size) # skip initial kernel_size of padding
    ix = 0
    if self.returnSurroundingIndices:
      surrounding_indices = [-1, 0, 1]
      ret_arr = torch.empty((27, *self.kernel_size))
    else:
      surrounding_indices = [0]
      ret_arr = torch.empty((1, *self.kernel_size))
    for z in surrounding_indices:
      for y in surrounding_indices:
        for x in surrounding_indices:
          tmp_coords = np.add(arr_coord, np.multiply([x, y, z], self.kernel_size))
          # print(ix, tmp_coords, z, y, x)
          ret_arr[ix] = self.get_sub_matrix_from_coords(tmp_coords)
          ix += 1
    return ret_arr

  def get_sub_matrix_from_coords(self, coords):
    batch_start = coords
    # print(batch_start)
    batch_end = np.add(coords, self.kernel_size)
    # print(batch_end)
    ret_arr = self.samples[batch_start[0]:batch_end[0], batch_start[1]:batch_end[1], batch_start[2]:batch_end[2]]
    # print(ret_arr.shape)
    return ret_arr

  def __len__(self):
    return self.length

  def shuffle(self):
    pass

#### Fit function:

Define the fit function:

In [25]:
from torch.optim import SGD, Adam
from torch.nn import MSELoss, CrossEntropyLoss, L1Loss
import datetime

def fit(model, dset_train, num_epochs=1, train_batches=1, optimFunc=Adam, lossFunc=MSELoss, folder_path=folder_path, verbose=2):
  # with help from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

  curr_best_loss = np.Infinity
  training_losses = []
  save_path = ospath.join(folder_path, model.name)

  optimizer = optimFunc(model.parameters(), lr=0.1)
  loss = lossFunc()
  model.train(True)
  start_time = datetime.datetime.now()
  if (verbose > 0):
    print(f"starting training for model '{model.name}' at: {datetime.datetime.now()}")
  for i in range(num_epochs):
    t_losses = []

    for j in range(train_batches):
      x_train = dset_train[j]
      p = model(x_train)
      l = loss(p, x_train)

      optimizer.zero_grad()
      l.backward()
      optimizer.step()

      t_losses.append(l.item())

    curr_loss = np.mean(t_losses)
    training_losses.append(curr_loss)

    if curr_loss < curr_best_loss:
      curr_best_loss = curr_loss
      torch.save(model.state_dict(), save_path)
    if (verbose > 1):
      print(f"epoch {i+1}/{num_epochs} at time {datetime.datetime.now().time()}; current loss: {curr_loss}")
    elif (verbose == 1):
      print(f"epoch {i+1}/{num_epochs: <{10}}", end="\r")

  end_time = datetime.datetime.now()
  if (verbose > 0):
    print(f"end of training {num_epochs} epochs: {datetime.datetime.now()}; elapsed time: {end_time - start_time}")
  best_model = model
  best_model.load_state_dict(torch.load(save_path))

  return best_model, training_losses

#### Linear models:

##### AELinPass  
Try a pass linear model to make sure it works

In [26]:
class autoEncLinPass(nn.Module):

  def __init__(self, name="linae1", kernel_size=[8,8,8]):
    super(autoEncLinPass, self).__init__()
    sample_size = np.prod(kernel_size)
    self.encoder = nn.Sequential(
      nn.Flatten(),
      )
    self.decoder = nn.Sequential(
      nn.Unflatten(1, [1, *kernel_size]),
      )
    self.name = name

  def get_encoding(self, x):
    return self.encoder(x)

  def get_decoding(self, c):
    return self.decoder(c)

  def forward(self, x):
    c = self.encoder(x)
    y = self.decoder(c)
    return y

In [27]:
def trainAndOutputAutoEncPassModel(folder_path=folder_path, model=autoEncLinPass, kernel_size=[4,4,4], stride_size=[4,4,4], epochs=4, name="model1lin", vol_name="tooth", verbose=True):
  # to train model
  tooth_path=ospath.join(folder_path, files[vol_name]["file_name"])
  tooth_array=readRawFile(tooth_path, files[vol_name]["dtype"], files[vol_name]["shape"], cutoffLow=(50,0))#, gaussianFilter=True)
  model1toothDset = Dataset(tooth_array, stride=stride_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1 = model(name=name, kernel_size=kernel_size)

  # to output encoded representation
  if (verbose):
    print(f"starting encoding at: {datetime.datetime.now()}")
  toothEncDset = Dataset(tooth_array, stride=kernel_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1_encoded_reps = np.array([model1.encoder(toothEncDset[x]).detach().numpy().flatten() for x in range(toothEncDset.length)])
  model1_encoded_reps.reshape(-1).tofile(ospath.join(folder_path, f"{model1.name}.enc"), format="%<f")

  # output reconstructed model
  if (verbose):
      print(f"starting decoding at: {datetime.datetime.now()}")
  model1_reconstructed = np.empty(toothEncDset.samples.shape)
  for ix, enc in enumerate(model1_encoded_reps):
    tmp_dec = model1.decoder(torch.tensor(enc).reshape((1,-1))).detach().numpy()
    ixst = np.multiply(np.unravel_index(ix, toothEncDset.unfold_dims), toothEncDset.stride)
    ixed = np.add(ixst,toothEncDset.kernel_size)
    # print(ix, ixst, ixed)
    model1_reconstructed[ixst[0]:ixed[0], ixst[1]:ixed[1], ixst[2]:ixed[2]] = tmp_dec

  model1_reconstructed = model1_reconstructed[0:toothEncDset.orig_shape[0], 0:toothEncDset.orig_shape[1], 0:toothEncDset.orig_shape[2]]
  # model1_reconstructed = gaussian_filter(model1_reconstructed, sigma=1, radius=np.divide(kernel_size,2).astype(np.int32)) # filter output to get rid of some of the blockiness
  mse = np.mean(np.power((model1_reconstructed - tooth_array),2)) # from https://stackoverflow.com/a/18047482
  print(f"MSE reconstructed - input: {mse}")
  model1_reconstructed = (model1_reconstructed*255).astype(np.uint8).transpose().reshape(-1)
  model1_reconstructed.tofile(ospath.join(folder_path, f"{model1.name}.raw"), format="%<hhf")

  return

In [28]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_name = "linae1pass_8_8"
model1 = autoEncLinPass

model1_epochs = 0 # 0 epochs just runs the encoder/decoder without training

# trainAndOutputAutoEncPassModel(folder_path=folder_path, model=model1, kernel_size=model1_kernel, stride_size=model1_stride, epochs=model1_epochs, name=model1_name, vol_name="body", verbose=True)

##### AELin1

In [29]:
class autoEncLin1(nn.Module):

  def __init__(self, name="linae1", kernel_size=[8,8,8], enc_size=9):
    super(autoEncLin1, self).__init__()
    sample_size = np.prod(kernel_size)
    self.encoder = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=sample_size, out_features=int(sample_size/2)),
      nn.PReLU(), #use parametric relu for activation functions
      nn.Linear(int(sample_size/2), int(sample_size/4)),
      nn.PReLU(),
      nn.Linear(int(sample_size/4), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), enc_size),
      nn.PReLU(),
      )
    self.decoder = nn.Sequential(
      nn.Linear(enc_size, int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size/4)),
      nn.PReLU(),
      nn.Linear(int(sample_size/4), int(sample_size/2)),
      nn.PReLU(),
      nn.Linear(int(sample_size/2), int(sample_size)),
      nn.Unflatten(1, kernel_size),
      nn.Sigmoid(),
      )
    self.name = name

  def get_encoding(self, x):
    return self.encoder(x)

  def get_decoding(self, c):
    return self.decoder(c)

  def forward(self, x):
    c = self.encoder(x)
    y = self.decoder(c)
    return y

In [30]:
def trainAndOutputAutoEnc1Model(folder_path=folder_path, model=autoEncLin1, kernel_size=[4,4,4], stride_size=[4,4,4], enc_size=4, epochs=4, name="model1lin", optim=SGD, loss=MSELoss, vol_name="tooth", verbose=0, load_model_weights=False):
  # to train model
  tooth_path=ospath.join(folder_path, files[vol_name]["file_name"])
  tooth_array=readRawFile(tooth_path, files[vol_name]["dtype"], files[vol_name]["shape"])#, cutoffLow=(50,0))#, gaussianFilter=True)
  model1toothDset = Dataset(tooth_array, stride=stride_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1 = model(name=name, kernel_size=kernel_size, enc_size=enc_size)
  if (load_model_weights):
    save_path = ospath.join(folder_path, model1.name)
    model1.load_state_dict(torch.load(save_path))
  model1_best, model1_losses = fit(model1, model1toothDset, num_epochs=epochs, train_batches=model1toothDset.length-1, optimFunc=optim, lossFunc=loss, folder_path=folder_path, verbose=verbose)

  # to output encoded representation
  if (verbose > 0):
    print(f"starting encoding at: {datetime.datetime.now()}")
  toothEncDset = Dataset(tooth_array, stride=kernel_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1_encoded_reps = np.array([model1_best.encoder(toothEncDset[x]).detach().numpy().flatten().astype(np.float16) for x in range(toothEncDset.length)])
  model1_encoded_reps.reshape(-1).tofile(ospath.join(folder_path, f"{model1.name}.enc"), format="%<hf")

  # output reconstructed model
  if (verbose>0):
      print(f"starting decoding at: {datetime.datetime.now()}")
  model1_reconstructed = np.empty(toothEncDset.samples.shape)
  for ix, enc in enumerate(model1_encoded_reps):
    tmp_dec = model1_best.decoder(torch.tensor(enc.astype(np.float32)).reshape((1,-1))).detach().numpy()
    ixst = np.multiply(np.unravel_index(ix, toothEncDset.unfold_dims), toothEncDset.stride)
    ixed = np.add(ixst,toothEncDset.kernel_size)
    # print(ix, ixst, ixed)
    model1_reconstructed[ixst[0]:ixed[0], ixst[1]:ixed[1], ixst[2]:ixed[2]] = tmp_dec
  if (verbose>0):
      print(f"finish decoding at: {datetime.datetime.now()}")

  model1_reconstructed = model1_reconstructed[0:toothEncDset.orig_shape[0], 0:toothEncDset.orig_shape[1], 0:toothEncDset.orig_shape[2]]
  # model1_reconstructed = gaussian_filter(model1_reconstructed, sigma=1, radius=np.divide(kernel_size,2).astype(np.float32)) # filter output to get rid of some of the blockiness
  mse = np.mean((model1_reconstructed - tooth_array)**2) # from https://stackoverflow.com/a/18047482
  mae = np.mean(np.abs((model1_reconstructed - tooth_array)))
  print(f"MSE reconstructed - input: {mse}; MAE: {mae}")
  model1_reconstructed = (model1_reconstructed*255).astype(np.uint8).transpose().reshape(-1)
  model1_reconstructed.tofile(ospath.join(folder_path, f"{model1.name}.raw"), format="%<hhf")

  model1_decoder = model1_best
  model1_decoder.encoder = None
  torch.save(model1_decoder.state_dict(), ospath.join(folder_path, f"{model1.name}.dec"))

  return model1_losses

In [31]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_enc_size = 8
model1_name = "linae1_8_8_8"
model1 = autoEncLin1

model1_epochs = 1 # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = False # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

# model1_losses = trainAndOutputAutoEnc1Model(folder_path, model1, model1_kernel, model1_stride, model1_enc_size, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="body", verbose=2, load_model_weights=model1_load_weights)
# pd.Series(model1_losses).describe()

##### AELin2

In [32]:
class autoEncLin2(nn.Module):

  def __init__(self, name="linae2", kernel_size=[8,8,8], enc_size=9):
    super(autoEncLin2, self).__init__()
    sample_size = np.prod(kernel_size)
    self.encoder = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=sample_size, out_features=int(sample_size/8)),
      nn.PReLU(), #use parametric relu for activation functions
      nn.Linear(int(sample_size/8), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size/12)),
      nn.PReLU(),
      nn.Linear(int(sample_size/12), enc_size),
      nn.PReLU(),
      )
    self.decoder = nn.Sequential(
      nn.Linear(enc_size, int(sample_size/12)),
      nn.PReLU(),
      nn.Linear(int(sample_size/12), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size)),
      nn.Unflatten(1, kernel_size),
      nn.Sigmoid(),
      )
    self.name = name

  def get_encoding(self, x):
    return self.encoder(x)

  def get_decoding(self, c):
    return self.decoder(c)

  def forward(self, x):
    c = self.encoder(x)
    y = self.decoder(c)
    return y

In [33]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_enc_size = 8
model1_name = f"linae2_8_8_{model1_enc_size}"
model1 = autoEncLin2

model1_epochs = 1 # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = False # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

# model1_losses = trainAndOutputAutoEnc1Model(folder_path, model1, model1_kernel, model1_stride, model1_enc_size, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="body", verbose=2, load_model_weights=model1_load_weights)
# pd.Series(model1_losses).describe()

#### Non-neural encoders (or mixed models)

Try a model that uses coordinates, variances and singular values

In [34]:
from torch.optim import SGD, Adam
from torch.nn import MSELoss, CrossEntropyLoss
import datetime

def fitWithIndex(model, dset_train, num_epochs=1, train_batches=1, optimFunc=Adam, lossFunc=MSELoss, folder_path=folder_path, verbose=2):
  # with help from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

  curr_best_loss = np.Infinity
  training_losses = []
  save_path = ospath.join(folder_path, model.name)

  optimizer = optimFunc(model.parameters(), lr=0.1)
  loss = lossFunc()
  model.train(True)
  start_time = datetime.datetime.now()
  if (verbose > 0):
    print(f"starting training for model '{model.name}' at: {datetime.datetime.now()}")
  for i in range(num_epochs):
    t_losses = []

    for j in range(train_batches):
      x_train = dset_train[j]
      p = model(x_train, j)
      l = loss(p, x_train)

      optimizer.zero_grad()
      l.backward()
      optimizer.step()

      t_losses.append(l.item())

    curr_loss = np.mean(t_losses)
    training_losses.append(curr_loss)

    if curr_loss < curr_best_loss:
      curr_best_loss = curr_loss
      torch.save(model.state_dict(), save_path)
    if (verbose > 1):
      print(f"epoch {i+1}/{num_epochs} at time {datetime.datetime.now().time()}; current loss: {curr_loss}")
    elif (verbose == 1):
      print(f"epoch {i+1}/{num_epochs: <{10}}", end="\r")

  end_time = datetime.datetime.now()
  if (verbose > 0):
    print(f"end of training {num_epochs} epochs: {datetime.datetime.now()}; elapsed time: {end_time - start_time}")
  best_model = model
  best_model.load_state_dict(torch.load(save_path))

  return best_model, training_losses

##### Mixmodel1

In [35]:
class MixedModel1(nn.Module):

  def get_encoding(self, x, index):
    arr_coord = np.multiply(np.unravel_index(index, self.dset.unfold_dims), self.dset.stride)+self.kernel_size # add one kernel_size to skip padded 0s
    ret_coords = np.divide(arr_coord, self.dset.samples.shape)
    retCodeInfo = torch.empty(9, dtype=torch.float32)
    # get normalized coords
    retCodeInfo[0:3] = torch.tensor(ret_coords[0:3], dtype=torch.float32)
    # get mean variances
    retCodeInfo[3] = torch.mean(torch.var(torch.reshape(x, self.kernel_size), axis=(0)))
    retCodeInfo[4] = torch.mean(torch.var(torch.reshape(x, self.kernel_size), axis=(1)))
    retCodeInfo[5] = torch.mean(torch.var(torch.reshape(x, self.kernel_size), axis=(2)))
    # get svds
    retCodeInfo[6:9] = torch.linalg.svdvals(torch.reshape(x, self.kernel_size))[0:3,0]
    #combine into ret_array

    return retCodeInfo.reshape((1,-1))

  def __init__(self, dset, name="mxae1"):
    super(MixedModel1, self).__init__()
    self.dset = dset
    kernel_size = dset.kernel_size
    self.kernel_size = kernel_size
    sample_size = np.prod(kernel_size)
    self.encoder = self.get_encoding
    self.decoder = nn.Sequential(
      nn.Linear(9, int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size/4)),
      nn.PReLU(),
      nn.Linear(int(sample_size/4), int(sample_size/2)),
      nn.PReLU(),
      nn.Linear(int(sample_size/2), int(sample_size)),
      nn.Unflatten(1, kernel_size),
      nn.Sigmoid(),
      )
    self.name = name


  def get_decoding(self, c):
    return self.decoder(c)

  def forward(self, x, i):
    c = self.encoder(x, i)
    y = self.decoder(c)
    return y

In [45]:
def trainAndOutputMixedModel(folder_path=folder_path, model=MixedModel1, kernel_size=[4,4,4], stride_size=[4,4,4], epochs=4, name="model1mix", optim=SGD, loss=MSELoss, vol_name="tooth", verbose=True, load_model_weights=False):
  # to train model
  tooth_path=ospath.join(folder_path, files[vol_name]["file_name"])
  tooth_array=readRawFile(tooth_path, files[vol_name]["dtype"], files[vol_name]["shape"])#, cutoffLow=(50, 0))
  model1toothDset = Dataset(tooth_array, stride=stride_size, kernel_size=kernel_size, returnSurroundingIndices=False)

  model1 = model(dset=model1toothDset, name=name)
  if (load_model_weights):
    save_path = ospath.join(folder_path, model1.name)
    model1.load_state_dict(torch.load(save_path))
  model1_best, model1_losses = fitWithIndex(model1, model1toothDset, num_epochs=epochs, train_batches=model1toothDset.length-1, optimFunc=optim, lossFunc=loss, folder_path=folder_path, verbose=verbose)

  # to output encoded representation
  if (verbose):
    print(f"starting encoding at: {datetime.datetime.now()}")
  toothEncDset = Dataset(tooth_array, stride=kernel_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1_encoded_reps = np.array([model1_best.encoder(toothEncDset[x], x).detach().numpy().flatten().astype(np.float16) for x in range(toothEncDset.length)])
  model1_encoded_reps.reshape(-1).tofile(ospath.join(folder_path, f"{model1.name}.enc"), format="%<hf")

  # output reconstructed model
  if (verbose):
      print(f"starting decoding at: {datetime.datetime.now()}")
  model1_reconstructed = np.empty(toothEncDset.samples.shape)
  for ix, enc in enumerate(model1_encoded_reps):
    tmp_dec = model1_best.decoder(torch.tensor(enc, dtype=torch.float32).reshape((1,-1))).detach().numpy()
    ixst = np.multiply(np.unravel_index(ix, toothEncDset.unfold_dims), toothEncDset.stride)
    ixed = np.add(ixst,toothEncDset.kernel_size)
    # print(ix, ixst, ixed)
    model1_reconstructed[ixst[0]:ixed[0], ixst[1]:ixed[1], ixst[2]:ixed[2]] = tmp_dec
  if (verbose>0):
      print(f"finish decoding at: {datetime.datetime.now()}")

  model1_reconstructed = model1_reconstructed[0:toothEncDset.orig_shape[0], 0:toothEncDset.orig_shape[1], 0:toothEncDset.orig_shape[2]]
  model1_reconstructed = gaussian_filter(model1_reconstructed, sigma=1, radius=np.divide(kernel_size,2).astype(np.int32)) # filter output to get rid of some of the blockiness
  mse = np.mean((model1_reconstructed - tooth_array)**2) # from https://stackoverflow.com/a/18047482
  mae = np.mean(np.abs((model1_reconstructed - tooth_array)))
  print(f"MSE reconstructed - input: {mse}; MAE: {mae}")
  model1_reconstructed = (model1_reconstructed*255).astype(np.uint8).transpose().reshape(-1)
  model1_reconstructed.tofile(ospath.join(folder_path, f"{model1.name}.raw"), format="%<hhf")

  return model1_losses

In [46]:
model1_kernel = [8]*3
model1_stride = model1_kernel
model1_name = "coordvarsvd1_8_8"
model1 = MixedModel1

model1_epochs = 0 # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = False # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

model1_losses = trainAndOutputMixedModel(folder_path, model1, model1_kernel, model1_stride, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="body", verbose=2, load_model_weights=model1_load_weights)
pd.Series(model1_losses).describe()

starting training for model 'coordvarsvd1_8_8' at: 2024-05-26 20:34:51.623919
end of training 0 epochs: 2024-05-26 20:34:51.624090; elapsed time: 0:00:00.000176
starting encoding at: 2024-05-26 20:34:51.634111
starting decoding at: 2024-05-26 20:35:44.717749
finish decoding at: 2024-05-26 20:36:28.882321
MSE reconstructed - input: 0.027095272276806626; MAE: 0.06434224731624635


count       0
unique      0
top       NaN
freq      NaN
dtype: object

##### Mixmodel3

In [38]:
class MixedModel3(nn.Module):

  def get_encoding(self, x, index):
    arr_coord = np.multiply(np.unravel_index(index, self.dset.unfold_dims), self.dset.stride)+self.kernel_size # add one kernel_size to skip padded 0s
    ret_coords = np.divide(arr_coord, self.dset.samples.shape)
    retCodeInfo = torch.empty(3, dtype=torch.float32)
    retCodeInfo = torch.linalg.svdvals(torch.reshape(x, self.kernel_size))[0:3,0]

    return retCodeInfo.reshape((1,-1))

  def __init__(self, dset, name="mxae3"):
    super(MixedModel3, self).__init__()
    self.dset = dset
    kernel_size = dset.kernel_size
    self.kernel_size = kernel_size
    sample_size = np.prod(kernel_size)
    self.encoder = self.get_encoding
    self.decoder = nn.Sequential(
      nn.Linear(3, int(sample_size/12)),
      nn.PReLU(),
      nn.Linear(int(sample_size/12), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size)),
      nn.Unflatten(1, kernel_size),
      nn.Sigmoid(),
      )
    self.name = name


  def get_decoding(self, c):
    return self.decoder(c)

  def forward(self, x, i):
    c = self.encoder(x, i)
    y = self.decoder(c)
    return y

In [39]:
model1_kernel = [4]*3
model1_stride = model1_kernel
model1_name = "svd2_4_4"
model1 = MixedModel3

model1_epochs = 1 # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = False # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

# model1_losses = trainAndOutputMixedModel(folder_path, model1, model1_kernel, model1_stride, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="body", verbose=2, load_model_weights=model1_load_weights)
# pd.Series(model1_losses).describe()